In [23]:
import pandas as pd
import re
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from tqdm import trange
import time
import string
import re
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import pymysql
pymysql.install_as_MySQLdb()
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import xlrd
# from fuzzywuzzy import fuzz
import re
from collections import Counter
from tqdm import tqdm
import time
import rapidfuzz
from fuzzywuzzy import process, fuzz

/Users/shenzongqi/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
wikipedia_table = pd.read_csv('/Users/shenzongqi/Downloads/WikipediaSchools.csv')
wikipedia_table['schoolURL'] = None
wikipedia_table

,arcoName,city,state,schoolURL
0,Boston University,Boston,Massachusetts,None
1,Brandeis University,Waltham,Massachusetts,None
2,Brown University,Providence,Rhode Island,None
3,California Institute of Technology,Pasadena,California,None
4,Carnegie Mellon University,Pittsburgh,Pennsylvania,None
...,...,...,...,...
1777,Highline College,Des Moines,Washington,None
1778,Pierce College,Lakewood,Washington,None
1779,Wisconsin Technical College System,None,None,None
1780,Wisconsin Technical College System,None,None,None


In [ ]:
string = "Hey! What's up bro?"
new_string = re.sub(r"[^a-zA-Z0-9]","",string)
print(new_string)

In [12]:
def drop_pun(str):
#     for i in item:
#         if i in string.punctuation:
#             item = re.sub(i,'',item)
#     return item
    str = re.sub(r"[^a-zA-Z0-9]","",str)
    return str

def to_lower(item):
    return item.lower()

#
def stringCleaning(str):
    if str is None:
        return None
    try:
        str = drop_pun(str)
        str = to_lower(str)
        return str
    except:
        return None

In [13]:
stringCleaning('ab.     !cIKL.`')

'abcikl'

In [14]:
def DataClean(table, column1, column2, column3, column4, newcolumn1, newcolumn2, newcolumn3, newcolumn4):
    for i in range(len(table)):
        table.loc[i, newcolumn1] = stringCleaning(table[column1][i])
        table.loc[i, newcolumn2] = stringCleaning(table[column2][i])
        table.loc[i, newcolumn3] = stringCleaning(table[column3][i])
        table.loc[i, newcolumn4] = stringCleaning(table[column4][i])

In [15]:
DataClean(wikipedia_table, 'arcoName', 'city', 'state', 'schoolURL', 'Tb2_arcoName', 'Tb2_city', 'Tb2_state', 'Tb2_schoolURL')
wikipedia_table

,arcoName,city,state,schoolURL,Tb2_arcoName,Tb2_city,Tb2_state,Tb2_schoolURL
0,Boston University,Boston,Massachusetts,None,bostonuniversity,boston,massachusetts,NaN
1,Brandeis University,Waltham,Massachusetts,None,brandeisuniversity,waltham,massachusetts,NaN
2,Brown University,Providence,Rhode Island,None,brownuniversity,providence,rhodeisland,NaN
3,California Institute of Technology,Pasadena,California,None,californiainstituteoftechnology,pasadena,california,NaN
4,Carnegie Mellon University,Pittsburgh,Pennsylvania,None,carnegiemellonuniversity,pittsburgh,pennsylvania,NaN
...,...,...,...,...,...,...,...,...
1777,Highline College,Des Moines,Washington,None,highlinecollege,desmoines,washington,NaN
1778,Pierce College,Lakewood,Washington,None,piercecollege,lakewood,washington,NaN
1779,Wisconsin Technical College System,None,None,None,wisconsintechnicalcollegesystem,none,none,NaN
1780,Wisconsin Technical College System,None,None,None,wisconsintechnicalcollegesystem,none,none,NaN


In [16]:
wikipedia_table.to_csv('./temp.csv', index = False)

In [24]:
scorer_dict = { 'R':fuzz.ratio, 
                'PR': fuzz.partial_ratio, 
                'TSeR': fuzz.token_set_ratio, 
                'TSoR': fuzz.token_sort_ratio,
                'PTSeR': fuzz.partial_token_set_ratio, 
                'PTSoR': fuzz.partial_token_sort_ratio, 
                'WR': fuzz.WRatio, 
                'QR': fuzz.QRatio,
                'UWR': fuzz.UWRatio, 
                'UQR': fuzz.UQRatio }

In [25]:
scorer_dict

{'R': <function fuzzywuzzy.fuzz.ratio(s1, s2)>,
 'PR': <function fuzzywuzzy.fuzz.partial_ratio(s1, s2)>,
 'TSeR': <function fuzzywuzzy.fuzz.token_set_ratio(s1, s2, force_ascii=True, full_process=True)>,
 'TSoR': <function fuzzywuzzy.fuzz.token_sort_ratio(s1, s2, force_ascii=True, full_process=True)>,
 'PTSeR': <function fuzzywuzzy.fuzz.partial_token_set_ratio(s1, s2, force_ascii=True, full_process=True)>,
 'PTSoR': <function fuzzywuzzy.fuzz.partial_token_sort_ratio(s1, s2, force_ascii=True, full_process=True)>,
 'WR': <function fuzzywuzzy.fuzz.WRatio(s1, s2, force_ascii=True, full_process=True)>,
 'QR': <function fuzzywuzzy.fuzz.QRatio(s1, s2, force_ascii=True, full_process=True)>,
 'UWR': <function fuzzywuzzy.fuzz.UWRatio(s1, s2, full_process=True)>,
 'UQR': <function fuzzywuzzy.fuzz.UQRatio(s1, s2, full_process=True)>}